In [4]:
# https://colab.research.google.com/notebooks/welcome.ipynb#scrollTo=P-H6Lw1vyNNd
%config IPCompleter.greedy=True
%matplotlib inline

# Import the dependencies.
import matplotlib.pyplot as plt # Dataset visualization.
import json
import numpy as np
import pandas as pd

# https://towardsdatascience.com/master-the-coco-dataset-for-semantic-image-segmentation-part-1-of-2-732712631047
import cv2
import cvlib as cv
# print(tensorflow.__version__)

# from config import IFNEEDED

In [7]:
# Look into this
#https://www.tensorflow.org/hub/tutorials/tf2_image_retraining <---

# cvlib it’s a high level library that runs object detection with just a few lines of code; it uses OpenCV and TensorFlow under the hood. We don’t even need to train a model our-self: cvlib uses a model pre-trained on the COCO dataset, capable of detecting 80 common objects. Which is totally sweet!

# I'm going to load the cvlib library and the YOLOv3 model, then detect the objects present in the test-image.jpeg image.

image = cv2.imread("../resources/test-image.jpeg")
boxes, labels, _conf = cv.detect_common_objects(image, model="yolov3")

# It prints the detected objects’ labels and bounding boxes at the end.
print(labels, boxes)

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
# I need to time things to figure out what is taking so long!

import time

start = time.time()
boxes, labels, _conf = cv.detect_common_objects(img, model="yolov3")
print("first detection: ", time.time() - start)

In [ ]:
# I know I will be sending binary data representing the image or frame of a video stream, so I just can't use the cvlib with image files but raw binary image data. So I need to be able to read that data from the CLI or WEB FRAMEWORK

# https://docs.python.org/3/library/sys.html but check this out https://docs.python.org/3/library/os.html#os.fdopen
import sys


# stdin stands for standard input which is a stream from which the program read its input data.
for line in sys.stdin:
    
    # expecting line in for of "num,num\n" so we want to strip whitespaces and newlines.
    line = line.strip()
    # EOF
    if line == "": break
    
    # convert string elements into integers and then sum them.
    values = line.split(",")
    nums = map(int, values)
    result = sum(nums)

    # send the result via stdout and back out as a string.
    sys.stdout.write(str(result) + "\n")
    sys.stdout.flush()

    # this is important becasue I see myself using Celery with RabbitMQ to handle all the incoming requests. https://docs.celeryproject.org/en/stable/getting-started/introduction.html

In [ ]:
# future work ...
# def read_msg ...
# def detect ...
# def write_result ...

# Since I'm dealing with sending binary data I could attach an ID to the string to note which image is coming from where.

In [ ]:
# encode the result into a json string and we write to output_f the message’s total size total_msg_size
from struct import unpack, pack

def write_result(output, image_id, shape, boxes, labels):
    result = json.dumps({
        'shape': shape,
        'boxes': boxes, 
        'labels': labels
    }).encode("ascii")

    # Use a preappended uuid to the image string to indentify
    total_msg_size = len(result) + UUID4_SIZE

    header = pack("!I", total_msg_size)
    output.write(header)
    output.write(image_id)
    output.write(result)
    output.flush()
